<a href="https://colab.research.google.com/github/mattanova/ELE6310E-Assignments/blob/main/HW2_Q3_MN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
import sys
import os
import shutil
import warnings

# Install requirements
!pip install -qr /content/gdrive/MyDrive/ELE6310E/A2/requirements.txt
# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')

In [ ]:
#installing timeloop/accelergy
!source /content/gdrive/MyDrive/ELE6310E/A2/install_timeloop/install_timeloop.sh
!export PATH=$PATH:~/.local/bin
!cp ~/.local/bin/timeloop* /usr/bin/
!cp ~/ENV/bin/accelergy* /usr/bin/

In [ ]:
%cd /content/gdrive/MyDrive/ELE6310E/A2

In [5]:
import solution
import torch
import numpy as np
import random
from matplotlib import pyplot as plt
import os

In [6]:
from common.utils import load_CIFAR10_dataset, evaluate, fit, model_size
from common.resnet import resnet32

In [7]:
#initializing the model
Seed = 6310
torch.manual_seed(Seed)
np.random.seed(Seed)
random.seed(Seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Seed)
    torch.cuda.manual_seed_all(Seed)

In [ ]:
train_loader, test_loader, calibration_loader = load_CIFAR10_dataset(batch_size=256, calibration_batch_size=1024)
model = resnet32(pretrained=True, save_path='./save/')
device = torch.device('cuda:0')
model.to(device)

accuracy = evaluate(model, test_loader, device)
print("test accuracy of fp model:", accuracy, "model size:", model_size(model))

In [9]:
# defining the pruning ratio layer-wise
prune_ratio_dict = {'conv1': 0.4440841438747278,
 'layer1.0.conv1': 0.0,
 'layer1.0.conv2': 0.21017264220055182,
 'layer1.1.conv1': 0.0,
 'layer1.1.conv2': 0.1304443521010617,
 'layer1.2.conv1': 0.2179308491794905,
 'layer1.2.conv2': 0.49600963729509406,
 'layer1.3.conv1': 0.2676747798862238,
 'layer1.3.conv2': 0.1145834621899427,
 'layer1.4.conv1': 0.0,
 'layer1.4.conv2': 0.31566235085885297,
 'layer2.0.conv1': 0.0,
 'layer2.0.conv2': 0.0,
 'layer2.0.downsample.0': 0.5,
 'layer2.1.conv1': 0.0,
 'layer2.1.conv2': 0.3018744694243821,
 'layer2.2.conv1': 0.5,
 'layer2.2.conv2': 0.47850360367177003,
 'layer2.3.conv1': 0.0,
 'layer2.3.conv2': 0.2827091030918264,
 'layer2.4.conv1': 0.2883387129263232,
 'layer2.4.conv2': 0.22498356321872848,
 'layer3.0.conv1': 0.0,
 'layer3.0.conv2': 0.0,
 'layer3.0.downsample.0': 0.6777966477707704,
 'layer3.1.conv1': 0.0,
 'layer3.1.conv2': 0.0,
 'layer3.2.conv1': 0.0,
 'layer3.2.conv2': 0.2311303372748441,
 'layer3.3.conv1': 0.0,
 'layer3.3.conv2': 0.24996009553444842,
 'layer3.4.conv1': 0.0,
 'layer3.4.conv2': 0.7088984873721569}

sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
accuracy = evaluate(sparsed_model, test_loader, device)
print("test accuracy after pruning:", accuracy, "model size:", model_size(sparsed_model))

test accuracy after pruning: 85.2199935913086 model size: model size: 1.520MB


In [ ]:
# a heuristic search for possible pruning ratio encrementation
import random
for key,value in prune_ratio_dict.items():
    rand = random.uniform(0.1,0.3) # this range can be changed, for ex. wider for initial times searching
    prune_ratio_dict[key] += rand
    sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
    temp_accuracy = evaluate(sparsed_model, test_loader, device)
    if accuracy - temp_accuracy > accuracy*0.1: # to skip layers have a significant effect
            prune_ratio_dict[key] -= rand
    elif temp_accuracy < 85:    # accuracy constraint
        prune_ratio_dict[key] -= rand
    elif temp_accuracy > 86:    # to allow pruning as much as possible
        accuracy = temp_accuracy
    else:
        break
print("pruned model accuracy is: ",accuracy)

In [10]:
sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
accuracy = evaluate(sparsed_model, test_loader, device)
print("test accuracy after pruning:", accuracy, "model size:", model_size(sparsed_model))

test accuracy after pruning: 85.2199935913086 model size: model size: 1.520MB


In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(sparsed_model.parameters(), 1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)
scheduler = None
train_accuracy, test_accuracy = fit(sparsed_model, 5, train_loader, test_loader, criterion, optimizer, scheduler, device)

fit:  20%|██        | 1/5 [00:50<03:23, 50.80s/it]

epoch 1: train_accuracy=99.1839828491211, test_accuracy=92.00999450683594


fit:  40%|████      | 2/5 [01:42<02:34, 51.53s/it]

epoch 2: train_accuracy=99.45598602294922, test_accuracy=92.39999389648438


fit:  60%|██████    | 3/5 [02:33<01:42, 51.03s/it]

epoch 3: train_accuracy=99.56847381591797, test_accuracy=92.58999633789062


fit:  80%|████████  | 4/5 [03:24<00:51, 51.10s/it]

epoch 4: train_accuracy=99.61756134033203, test_accuracy=92.66999816894531


epoch 5: train_accuracy=99.65437316894531, test_accuracy=92.72999572753906


In [12]:
solution.generate_resnet_layers(sparsed_model, base_path='common/layer_prob_base.yaml',  path='Q3/prob')

In [ ]:
energy_data = []
from pathlib import Path
problem_directory = 'Q3/prob'
yaml_files = list(Path(problem_directory).glob('*.yaml'))
for yaml_file in yaml_files:
    layer_name = yaml_file.stem
    print(yaml_file)
    !timeloop-mapper Q3/arch/eyeriss_like.yaml {yaml_file} Q3/mapper/mapper.yaml
    energy = solution.Run_Accelergy(path_to_eyeriss_files='Q3')
    energy_data.append((layer_name,energy))
    print(f'Layer: {layer_name}, Energy: {energy} uJ')

Q3/prob/layer1.3.conv1.yaml.yaml
input file: Q3/arch/eyeriss_like.yaml
input file: Q3/prob/layer1.3.conv1.yaml.yaml
input file: Q3/mapper/mapper.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/bin/accelergy Q3/arch/eyeriss_like.yaml Q3/prob/layer1.3.conv1.yaml.yaml Q3/mapper/mapper.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 8
Mapper configuration complete.
Initializing Index Factorization subspace.
  Factorization options along problem dime